In [1]:
from fastai.text.all import *
path = Path("data")

In [ ]:
files = get_text_files(path, folders = ['train', 'test'])
txts = L(["Fourscore and seven years ago, our fathers brought forth upon this continent a new nation, conceived in liberty, and dedicated to the pursuit of happiness.",
          "No, you don't understand!"])
txt = txts[0]

In [ ]:
tkn = Tokenizer(WordTokenizer())
toks = tkn(txt)
print(coll_repr(toks, 30))

In [ ]:
toks_all = txts.map(tkn)
toks_all

In [ ]:
num = Numericalize(min_freq=1)
num.setup(toks_all)
print(coll_repr(num.vocab,20))

In [ ]:
nums = num(toks)[:20]; nums

In [ ]:
' '.join(num.vocab[o] for o in nums)

In [ ]:
nums_all = toks_all.map(num)

In [ ]:
dl = LMDataLoader(nums_all)

Creating the language model:

In [2]:
get_files = partial(get_text_files, folders=['train', 'test'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_files, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [3]:
dls_lm.show_batch(max_n=2)

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The ser

,text,text_
0,"xxbos xxmaj we discuss here some of the uncertainties associated with the partitioning of xxup rs obtained using our method . xxbos xxmaj however , xxmaj request ( xxunk ) is not feasible because no path can support it . xxbos xxmaj so ah , how come you did n't call xxmaj jerry about all this ? xxbos xxmaj he 's gon na send a black and white to pick us up . xxbos xxmaj her parents wo nt let","xxmaj we discuss here some of the uncertainties associated with the partitioning of xxup rs obtained using our method . xxbos xxmaj however , xxmaj request ( xxunk ) is not feasible because no path can support it . xxbos xxmaj so ah , how come you did n't call xxmaj jerry about all this ? xxbos xxmaj he 's gon na send a black and white to pick us up . xxbos xxmaj her parents wo nt let her"
1,". xxbos xxmaj this is further supported by the band around xxunk - xxunk in the metal complexes due to xxunk ) and is xxunk with the appearance of new xxunk to medium bands in the region xxunk - xxunk , which could be assigned to the xxunk frequencies of xxunk ) bands , respectively , confirmed that , the xxunk to the metal ions is achieved xxunk - xxunk atoms [ 41 ] . xxbos xxmaj however , xxup","xxbos xxmaj this is further supported by the band around xxunk - xxunk in the metal complexes due to xxunk ) and is xxunk with the appearance of new xxunk to medium bands in the region xxunk - xxunk , which could be assigned to the xxunk frequencies of xxunk ) bands , respectively , confirmed that , the xxunk to the metal ions is achieved xxunk - xxunk atoms [ 41 ] . xxbos xxmaj however , xxup xxunk"


Fine-tuning the language model:

In [4]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [5]:
learn.fit_one_cycle(1, 2e-2)
learn.save_encoder('finetuned')

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.483150,4.052845,0.285742,57.560986,04:10


In [6]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [7]:
dls_clas.show_batch(max_n=3)

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/fastai/torch_core.py:500: FutureWarning: The ser

,text,category
0,"xxbos xxmaj using xxmaj lemma 3 with xxunk = xxunk < xxunk / 2 = xxunk and xxunk = xxunk we see that the first lower bound is uniformly better ( larger ) than the second one , whereas again with xxmaj lemma 3 , neither of the upper bounds xxup gν+1 / 2,ν+1 / 2 and xxup gν , ν+2 is uniformly better ( smaller ) than the other : in fact , with xxunk - xxunk / xxunk - xxunk / 2 and xxunk / 2 , we get xxunk / 4 - 1 / xxunk + xxunk / 4 - 1 / xxunk / xxunk ) , so that gν , xxunk / 2,ν+1 / xxunk ) for 0 < s < t and xxup gν+1 / 2,ν+1 / xxunk , xxunk ) for s > t.",formal
1,"xxbos xxmaj the following primers were used : xxmaj xxunk / 4 : forward xxunk xxrep 5 g xxup xxunk xxrep 5 c xxup xxunk , reverse xxunk ; xxmaj sox17 : forward xxunk , reverse xxunk xxrep 3 a xxup xxunk ; xxunk - xxunk ( xxunk ): forward xxunk , reverse xxunk xxrep 3 a xxup xxunk ; xxunk ( xxunk ): forward xxunk xxrep 3 g xxup xxunk , reverse xxunk xxrep 3 t xxup xxunk xxrep 3 c xxup xxunk xxrep 4 t xxup xxunk ; xxup ttr : forward xxunk , reverse xxunk xxrep 3 c xxup xxunk xxup xxunk ; and xxup xxunk : forward xxunk xxrep 3 c xxup xxunk xxrep 4 a xxup xxunk , reverse xxunk",formal
2,"xxbos xxmaj the model solves the one - dimensional equations of motion for an escaping atmosphere composed of several neutral and ionized xxunk - xxunk xxunk is the density of species s , v is the vertical velocity , xxmaj fs is the xxunk flux of species s , xxmaj xxunk is the net chemical source term for species s , xxunk is a force term xxunk from viscous acceleration , e = xxunk is the specific internal energy of the gas , xxup xxunk is the specific net radiative heating rate , xxunk is the coefficient of heat xxunk , and xxunk is the viscous xxunk functional ( e.g. , xxunk and xxmaj xxunk , 1989 ) .",formal


In [8]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [9]:
learn = learn.load_encoder('finetuned')

In [10]:
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,time
0,0.472082,0.499005,0.653000,02:04


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

epoch,train_loss,valid_loss,accuracy,time
0,0.396922,0.091640,0.990500,02:11


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

epoch,train_loss,valid_loss,accuracy,time
0,0.297270,0.041488,0.991500,03:07


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

epoch,train_loss,valid_loss,accuracy,time
0,0.247094,0.028939,0.990500,04:26
1,0.226380,0.020508,0.995000,04:16


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

In [28]:
learn.predict("Blessed are the meek, for they shall inherit the Earth.")

/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sivakalyan/Programming/Python/fastai_hackathon/.venv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


('informal', tensor(1), tensor([0.2006, 0.7994]))

In [29]:
save_pickle("formality.pkl", learn)